In [3]:
# install the CPTAC helper package

import importlib.util, sys, subprocess

pkg = "cptac"
if importlib.util.find_spec(pkg) is None:
    print("Installing cptac...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "cptac"])
else:
    print("cptac is already installed")


cptac is already installed


In [88]:
# load one CPTAC dataset

import cptac
# load one CPTAC datasets that got both proteomics and transcriptomics

import cptac
import pandas as pd

df = cptac.list_datasets()
print(df.head(10))


  Cancer            Source         Datatype
0   ucec               bcm       proteomics
1   ucec               bcm  transcriptomics
2   None  brca_mapping.csv             None
3   brca             broad  transcriptomics
4  ccrcc             broad  transcriptomics
5   coad             broad  transcriptomics
6    gbm             broad  transcriptomics
7  hnscc             broad  transcriptomics
8   lscc             broad  transcriptomics
9   luad             broad  transcriptomics


In [95]:
# Find the dataset with cancers, data source and the type of data

df2 = df.dropna(subset=["Cancer", "Source", "Datatype"]).copy()
print(df2.head(10))

# Count datatypes per (Cancer, Source)
counts = (
    df2.groupby(["Cancer", "Source"])["Datatype"]
       .value_counts()
       .unstack(fill_value=0)
)

print(counts.head(10))


# Keep only Cancer/Source pairs that have both proteomics and transcriptomics
both = counts[(counts.get("proteomics", 0) > 0) & (counts.get("transcriptomics", 0) > 0)]
print(both)

   Cancer Source         Datatype
0    ucec    bcm       proteomics
1    ucec    bcm  transcriptomics
3    brca  broad  transcriptomics
4   ccrcc  broad  transcriptomics
5    coad  broad  transcriptomics
6     gbm  broad  transcriptomics
7   hnscc  broad  transcriptomics
8    lscc  broad  transcriptomics
9    luad  broad  transcriptomics
10     ov  broad  transcriptomics
Datatype                CNV  acetylproteomics  ancestry_prediction  cibersort  \
Cancer      Source                                                              
all_cancers harmonized    0                 0                    1          0   
            mssm          0                 0                    0          0   
            washu         0                 0                    0          0   
brca        bcm           1                 0                    0          0   
            broad         0                 0                    0          0   
            umich         0                 2              